In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
for i in ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX']:
    print(f" select '{i}' as column_name, count(*) as cnt_nulls from boston where {i} is null union")

 select 'CRIM' as column_name, count(*) as cnt_nulls from boston where CRIM is null union
 select 'ZN' as column_name, count(*) as cnt_nulls from boston where ZN is null union
 select 'INDUS' as column_name, count(*) as cnt_nulls from boston where INDUS is null union
 select 'CHAS' as column_name, count(*) as cnt_nulls from boston where CHAS is null union
 select 'NOX' as column_name, count(*) as cnt_nulls from boston where NOX is null union


In [8]:
tasks+=1
pd.read_sql(
    ''' 
    select 'CRIM' as column_name, count(*) as cnt_nulls from boston where CRIM is null union
    select 'ZN' as column_name, count(*) as cnt_nulls from boston where ZN is null union
    select 'INDUS' as column_name, count(*) as cnt_nulls from boston where INDUS is null union
    select 'CHAS' as column_name, count(*) as cnt_nulls from boston where CHAS is null union
    select 'NOX' as column_name, count(*) as cnt_nulls from boston where NOX is null
    ''',
    con,
    
)

,column_name,cnt_nulls
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [9]:
tasks+=1
pd.read_sql(
    '''
    select 'CRIM' as column_name, count(distinct CRIM) as cnt_dist from boston union
    select 'ZN' as column_name, count(distinct ZN) as cnt_dist from boston union
    select 'INDUS' as column_name, count(distinct INDUS) as cnt_dist from boston union
    select 'CHAS' as column_name, count(distinct CHAS) as cnt_dist from boston union
    select 'NOX' as column_name, count(distinct NOX) as cnt_dist from boston 
    ''',
    con,
)

,column_name,cnt_dist
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [10]:
tasks+=1
pd.read_sql(
    '''select column_name,med from 
    (with a as
        (
            select CRIM as value , 'CRIM' as column_name, ROW_NUMBER() OVER(ORDER BY CRIM) ROW_N , min(CRIM) over() as min_val from boston union
            select ZN as value , 'ZN' as column_name, ROW_NUMBER() OVER(ORDER BY ZN) ROW_N , min(ZN) over() as min_val from boston union
            select INDUS as value , 'INDUS' as column_name, ROW_NUMBER() OVER(ORDER BY INDUS) ROW_N , min(INDUS) over() as min_val from boston union
            select CHAS as value , 'CHAS' as column_name, ROW_NUMBER() OVER(ORDER BY CHAS) ROW_N , min(CHAS) over() as min_val from boston union
            select NOX as value , 'NOX' as column_name, ROW_NUMBER() OVER(ORDER BY NOX) ROW_N , min(NOX) over() as min_val from boston
        )
        select distinct column_name,min_val,avg(value) over(partition by column_name) as med from a 
        where row_n in (253,254)) 
    where min_val = med
    ''',
    con,
)

# Не до конца понял цель задания. Вывел названия колонок, у который медиана = минимальное значение. Следовательно больше половины домов не граничат с рекой и 
# доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов. так же отсутствует
# 

,column_name,med
0,CHAS,0.0
1,ZN,0.0


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [11]:
tasks+=1
pd.read_sql(
    '''
    select 
        distinct criter,
        (select avg(rm) from boston where medv = (select max(medv) from boston)) avg_rm_max_medv,
        avg (rm) over(partition by criter) avg_rm_criter,
        (select avg(rm) from boston where medv = (select max(medv) from boston)) -avg (rm) over(partition by criter) dif_avg_rm 
        from (
            SELECT medv,rm, 
                case
                when row_number() over(ORDER BY MEDV) <= 25 then 'bot_25'
                when row_number() over(ORDER BY MEDV) <= 50 then 'bot_50'
                when row_number() over(ORDER BY MEDV) <= 100 then 'bot_100'
                when row_number() over(ORDER BY MEDV) <= 200 then 'bot_200'
                when row_number() over(ORDER BY MEDV) <= 300 then 'bot_300'
                else "dorogo_bogato" 
                end criter from boston
                )
    where criter !='dorogo_bogato'
    order by dif_avg_rm desc
    ''',
    con,
)

# влияние числа комнат очень значительное , 
#т.к разница между самым дорогоим сегментом и самым дешевым, всего 1.7 единицы

,criter,avg_rm_max_medv,avg_rm_criter,dif_avg_rm
0,bot_25,7.484,5.74784,1.73616
1,bot_50,7.484,5.75864,1.72536
2,bot_200,7.484,5.93629,1.54771
3,bot_100,7.484,6.02100,1.46300
4,bot_300,7.484,6.09327,1.39073


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [12]:
tasks+=1
pd.read_sql(
    '''select * from
    (
        select lstat,medv, rank() over (order by lstat) rank from boston
        )
    where medv = (select max(medv) from boston)
    ''',
    con,
)
# очевидно, что в самых дорогих домах процент насления с низким статус ниже,чем в других.
# Но нельзя сказать, что только в самых дорогих наименший процент такого населения]

,lstat,medv,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT distinct avg(MEDV) OVER (PARTITION BY CHAS) as 'MEDV_avg',
    CASE 
    WHEN CHAS = 1 then 'Граничит_с_рекой' 
    ELSE 'Не_граничит_с_рекой' 
    END as CHAS_type
    FROM boston
  
    ''',
    con,
)

,MEDV_avg,CHAS_type
0,22.093843,Не_граничит_с_рекой
1,28.440000,Граничит_с_рекой


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [14]:
for i in ['CRIM', 'ZN', 'INDUS', 'NOX']:
    print(f" select '{i}' as column_name count(1) from target where {i} > avg_{i} and chas = 1 group by {'column_name'} union")

 select 'CRIM' as column_name count(1) from target where CRIM > avg_CRIM and chas = 1 group by column_name union
 select 'ZN' as column_name count(1) from target where ZN > avg_ZN and chas = 1 group by column_name union
 select 'INDUS' as column_name count(1) from target where INDUS > avg_INDUS and chas = 1 group by column_name union
 select 'NOX' as column_name count(1) from target where NOX > avg_NOX and chas = 1 group by column_name union


In [15]:
# граничит с рекой
tasks+=1
pd.read_sql(
    '''
    with target as
        (
            select CRIM , avg(CRIM) over() avg_crim,ZN , avg(ZN) over() avg_zn,indus , avg(indus) over() avg_indus ,nox , avg(nox) over() avg_nox,chas from boston
            )
    select 'CRIM' as column_name, count(1) as cnt from target where CRIM > avg_CRIM and chas = 1 group by column_name union
    select 'ZN' as column_name, count(1) from target where ZN > avg_ZN and chas = 1 group by column_name union
    select 'INDUS' as column_name, count(1) from target where INDUS > avg_INDUS and chas = 1 group by column_name union
    select 'NOX' as column_name, count(1) from target where NOX > avg_NOX and chas = 1 group by column_name
    ''',
    con,
)

# '''Граница с рекой положительно влияет на уровень преступности, а так же на концетрацию озота
#   колличество объектов, где доля акров неторгового бизнеса больше средней почти в 11 раз меньше, чем когда не граничит с рекой
# Ситуций когда показатель ZN больше средней при границы с рекой в 18 раз меньше, чем когда нет Chas = 0

,column_name,cnt
0,CRIM,7
1,INDUS,18
2,NOX,15
3,ZN,7


In [16]:
# не граничит с рекой
tasks+=1
pd.read_sql(
    '''
    with target as
        (
            select CRIM , avg(CRIM) over() avg_crim,ZN , avg(ZN) over() avg_zn,indus , avg(indus) over() avg_indus ,nox , avg(nox) over() avg_nox,chas from boston
            )
    select 'CRIM' as column_name, count(1) as cnt from target where CRIM > avg_CRIM and chas = 0 group by column_name union
    select 'ZN' as column_name, count(1) from target where ZN > avg_ZN and chas = 0 group by column_name union
    select 'INDUS' as column_name, count(1) from target where INDUS > avg_INDUS and chas = 0 group by column_name union
    select 'NOX' as column_name, count(1) from target where NOX > avg_NOX and chas = 0 group by column_name
    ''',
    con,
)

,column_name,cnt
0,CRIM,120
1,INDUS,194
2,NOX,194
3,ZN,127


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [17]:
for i in (10,20,30,40,50,60,70,80,90,100):
    print(f'select avg(nox) as avg_nox ,avg(indus) as avg_indus , max(pers) as pers from table_1 where pers <={i} union')

select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=10 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=20 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=30 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=40 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=50 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=60 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=70 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=80 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=90 union
select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <

In [18]:
tasks+=1
pd.read_sql(
    ''' with table_1 as (
        SELECT nox, indus , ntile(100) over(order by nox,indus) as pers
        from boston
        )
     select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=10 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=20 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=30 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=40 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=50 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=60 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=70 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=80 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=90 union
    select avg(nox) avg_nox ,avg(indus) avg_indus , max(pers) as pers from table_1 where pers <=100
    ''',
    con,
)
# есть явная корреляция между Indus и NOX

,avg_nox,avg_indus,pers
0,0.409505,3.681964,10
1,0.421752,5.019151,20
2,0.432613,5.206923,30
3,0.447693,5.512379,40
4,0.462011,5.949258,50
5,0.476172,6.606176,60
6,0.492362,8.265225,70
7,0.509748,9.363374,80
8,0.529958,10.321338,90
9,0.554695,11.136779,100


In [19]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены не все задания! 🙀
